# L5: Information extraction

Information extraction (IE) is the task of identifying named entities and semantic relations between these entities in text data. In this lab we will focus on two sub-tasks in IE, **named entity recognition** (identifying mentions of entities) and **entity linking** (matching these mentions to entities in a knowledge base).

We start by loading spaCy:

In [1]:
import spacy

nlp = spacy.load("en_core_web_sm")

The data that we will be using has been tokenized following the conventions of the [Penn Treebank](ftp://ftp.cis.upenn.edu/pub/treebank/public_html/tokenization.html), and we need to prevent spaCy from using its own tokenizer on top of this. We therefore override spaCy&rsquo;s tokenizer with one that simply splits on space.

In [2]:
from spacy.tokens import Doc

class WhitespaceTokenizer(object):
    def __init__(self, vocab):
        self.vocab = vocab

    def __call__(self, text):
        return Doc(self.vocab, words=text.split(" "))

nlp.tokenizer = WhitespaceTokenizer(nlp.vocab)

## Data set

The main data set for this lab is a collection of news wire articles in which mentions of named entities have been annotated with page names from the [English Wikipedia](https://en.wikipedia.org/wiki/). The next code cell loads the training and the development parts of the data into Pandas data frames.

In [3]:
import bz2
import csv
import pandas as pd

with bz2.open("ner-train.tsv.bz2", 'rb') as source:
    df_train = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

with bz2.open("ner-dev.tsv.bz2", 'rb') as source:
    df_dev = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

Each row in these two data frames corresponds to one mention of a named entity and has five columns:

1. a unique identifier for the sentence containing the entity mention
2. the pre-tokenized sentence, with tokens separated by spaces
3. the start position of the token span containing the entity mention
4. the end position of the token span (exclusive, as in Python list indexing)
5. the entity label; either a Wikipedia page name or the generic label `--NME--`

The following cell prints the first five samples from the training data:

In [4]:
df_dev.head()

,sentence_id,sentence,beg,end,label
0,0946-000,CRICKET - LEICESTERSHIRE TAKE OVER AT TOP AFTE...,2,3,Leicestershire_County_Cricket_Club
1,0946-001,LONDON 1996-08-30,0,1,London
2,0946-002,West Indian all-rounder Phil Simmons took four...,0,2,West_Indies_cricket_team
3,0946-002,West Indian all-rounder Phil Simmons took four...,3,5,Phil_Simmons
4,0946-002,West Indian all-rounder Phil Simmons took four...,12,13,Leicestershire_County_Cricket_Club


In this sample, we see that the first sentence is annotated with three entity mentions:

* the span 0–1 &lsquo;EU&rsquo; is annotated as a mention but only labelled with the generic `--NME--`
* the span 2–3 &lsquo;German&rsquo; is annotated with the page [Germany](http://en.wikipedia.org/wiki/Germany)
* the span 6–7 &lsquo;British&rsquo; is annotated with the page [United_Kingdom](http://en.wikipedia.org/wiki/United_Kingdom)

## Problem 1: Evaluation measures

To warm up, we ask you to write code to print the three measures that you will be using for evaluation:

In [5]:
def evaluation_report(gold, pred):
    """Print precision, recall, and F1 score.
    
    Args:
        gold: The set with the gold-standard values.
        pred: The set with the predicted values.
    
    Returns:
        Nothing, but prints the precision, recall, and F1 values computed
        based on the specified sets.
    """
    # TODO: Replace the next line with your own code
    precision = len( gold.intersection(pred) )/len(pred)
    recall = len( gold.intersection(pred) )/len(gold)
    f1 = 2*(precision*recall)/(precision+recall)
    
    return({precision, recall, f1})

To test your code, you can run the following cell:

In [6]:
evaluation_report(set(range(3)), set(range(5)))

{0.6, 0.7499999999999999, 1.0}

This should give you a precision of 60%, a recall of 100%, and an F1-value of 75%.

## Problem 2: Span recognition

One of the first tasks that an information extraction system has to solve is to locate and classify (mentions of) named entities, such as persons and organizations. Here we will tackle the simpler task of recognizing **spans** of tokens that contain an entity mention, without the actual entity label.

The English language model in spaCy features a full-fledged [named entity recognizer](https://spacy.io/usage/linguistic-features#named-entities) that identifies a variety of entities, and can be updated with new entity types by the user. Your task in this problem is to evaluate the performance of this component when predicting entity spans in the development data.

Start by implementing a generator function that yields the gold-standard spans in a given data frame.

**Hint:** The Pandas method [`itertuples()`](https://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.DataFrame.itertuples.html) is useful when iterating over the rows in a DataFrame.

In [7]:
def gold_spans(df):
    """Yield the gold-standard mention spans in a data frame.

    Args:enen
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        triples consisting of the sentence id, start position, and end
        position of each span.
    """
    # TODO: Replace the next line with your own code
    for row in df.itertuples():
        yield (row[1],row[3],row[4])

To test your code, you can count the spans yielded by your function. When called on the development data, you should get a total of 5,917 unique triples. The first triple and the last triple should be

    ('0946-000', 2, 3)
    ('1161-010', 1, 3)  

In [29]:
spans_dev_gold = set(gold_spans(df_dev))
print(len(spans_dev_gold))

(list(gold_spans(df_dev))[0], list(gold_spans(df_dev))[-1])

5917


(('0946-000', 2, 3), ('1161-010', 1, 3))

Your next task is to write code that calls spaCy to predict the named entities in the development data, and to evaluate the accuracy of these predictions in terms of precision, recall, and F1. Print these scores using the function that you wrote for Problem&nbsp;1.

In [9]:
# TODO: Write code here to run and evaluate the spaCy NER on the development data

In [10]:
pred0 = list()
for row in df_dev.itertuples():
    doc = nlp(row[2])
    for ent in doc.ents:
        pred0.append(ent.text)

In [11]:
gold0 = list()
for i in gold_spans(df_dev):
    sen = (df_dev.loc[(df_dev['sentence_id'] == i[0]),'sentence'])
    words = list(sen[:1].str.split())[0]
    content = " ".join(words[i[1]:i[2]])
    gold0.append(content)

In [12]:
evaluation_report(set(gold0), set(pred0))

{0.5169769989047097, 0.585698774623856, 0.6754919499105546}

## Problem 3: Error analysis

As you were able to see in Problem&nbsp;2, the span accuracy of the named entity recognizer is far from perfect. In particular, only slightly more than half of the predicted spans are correct according to the gold standard. Your next task is to analyse this result in more detail.

Write code that prints the false positives and the false negatives from the automatic prediction. Have a look at the output. What are your observations? How could you improve the result? Discuss these questions in a short text.

In [13]:
# TODO: Write code here to do your analysis
uni = set(gold0).union(set(pred0))
fp = uni-set(gold0)
fn = uni-set(pred0)

In [14]:
list(fp)[:10]

['hundreds',
 '6 Lotte 5',
 '1998',
 'Baden',
 "last year 's",
 '11.27',
 '1,523',
 'July 31',
 '1982',
 '64.46']

In [15]:
list(fn)[:10]

['IDLC',
 'California Avocado Commission',
 'J. Russell',
 'Irish Republican Army',
 'MINSK',
 'GLASGOW',
 'Swansea',
 'Tarawa Amphibious Readiness Group',
 'Welsh',
 'SEATTLE']

*TODO: Write a short text that discusses the errors that you observed*

**Answer**:

`It seems that false negative includes many proper noun (NNP). On contact, the false postive includes many ordinal and measurable numbers, sentence parts beginning with small words and etc. Maybe removing such results as cardinal number (CD) and noun (NN, NNS) can improve the performance.`

Now, use the insights from your error analysis to improve the automated prediction that you implemented in Problem&nbsp;2. While the best way to do this would be to [update spaCy&rsquo;s NER model](https://spacy.io/usage/linguistic-features#updating) using domain-specific training data, for this lab it suffices to write code to pre-process the data and/or post-process the output produced by spaCy. You should be able to improve the F1 score from Problem&nbsp;2 by at last 15 percentage points.

In [92]:
# TODO: Write code here to improve the span prediction from Problem 2
pred01 = set(pred0)
gold01 = set(gold0)
for token in list(fp):
    t = nlp(token)
    for to in t:
        if to.tag_ in ["CD","NNS","EX","NN","SYM",
                       "RB","RBR","RBS","NFP",
                       "VB","POS","MD","DT","CC",
                       "JJ","JJS","JJR"]:
            pred01.discard(token)
evaluation_report(gold01, pred01)
# from 0.675 to 0.868

{0.6754919499105546, 0.7597585513078471, 0.8680459770114942}

In [114]:
# # TODO: Write code here to improve the span prediction from Problem 2
# import re
# def char_index(sentence, word_index):
#     sentence = re.split('(\s)',sentence) #Parentheses keep split characters
#     return len(''.join(sentence[:word_index*2]))

# train_data = list()
# for row in df_dev.itertuples():
#     sen = str(row[2])
#     start = char_index(sen, row[3])
#     end = char_index(sen, row[4])
#     new_tup = [start,end-1,row[5]]
#     train_data.append((row[2],new_tup))

In [116]:
# word = list()
# label = list()
# for i in train_data:
#     sen = i[0]
#     words = list(sen)
#     content = "".join(words[i[1][0]:i[1][1]])
#     word.append(content)
#     label.append(i[1][2])

In [51]:
# from spacy.gold import GoldParse
# import random
# optimizer = nlp.begin_training()
# for itn in range(100):
#     for i in range(len(word)):
#         doc = nlp.make_doc(word[i])
#         gold = GoldParse(doc, entities=label[i])
#         nlp.update([doc], [gold],drop=0.5, sgd=optimizer)

In [117]:
# from spacy.gold import GoldParse
# doc = Doc(nlp.vocab, word)
# gold = GoldParse(doc, label)
# optimizer = nlp.begin_training()
# nlp.update([doc], [gold], drop=0.5, sgd=optimizer)

Show that you achieve the performance goal by reporting the evaluation measures that you implemented in Problem&nbsp;1.

Before going on, we ask you to store the outputs of the improved named entity recognizer on the development data in a new data frame. This new frame should have the same layout as the original data frame for the development data that you loaded above, but should contain the predicted start and end positions for each token span. As the `label` of each span, you can use the special value `--NME--`.

In [158]:
# ws = list()
# for i in pred01:
#     ws = ws+i.split()

df = pd.DataFrame(columns = ["sentence_id", "sentence", "beg", "end","label"])
sent = df_dev.loc[:,['sentence_id','sentence']].drop_duplicates()    # 所有例句

for row in sent.itertuples():
    
#     doc = nlp(row[2])   # 检查所有的句子
#     for i in range(len(doc)):   
#         if str(doc[i]) in ws:    # 如果此单词在ws中
#             df.loc[df.shape[0]+1] = [row[1],row[2], i, i+1,"--NME--"]
#             aa = df["sentence_id"][df.shape[0]-1]   # 如果前一行的句子
#             bb = df["sentence_id"][df.shape[0]]     # 与当前新行句子相同
#             if aa == bb:
#                 if df["beg"][df.shape[0]] == df["end"][df.shape[0]-1]:  # 当前行的beg等于之前行的end
#                     df["end"][df.shape[0]-1] = df["end"][df.shape[0]-1]+1    # 上一行的end+1
#                     df.drop(index=df.shape[0],inplace=True)  # 删除当前行

    for gd in pred01:         # 所有筛选后的pred01
        docs = row[2].split()
        gds = gd.split()
        if (gd in row[2]) & (gds[0] in docs):
            beg = docs.index(gds[0])     # 找到第一次对应的单词
            end = beg+len(gds)           # 单词+词组长度=词组结尾位置
            df.loc[df.shape[0]+1] = [row[1], row[2], beg, end,"--NME--"]

In [159]:
print(len(df))
df[:10]

6923


,sentence_id,sentence,beg,end,label
1,0946-000,CRICKET - LEICESTERSHIRE TAKE OVER AT TOP AFTE...,0,1,--NME--
2,0946-001,LONDON 1996-08-30,0,1,--NME--
3,0946-002,West Indian all-rounder Phil Simmons took four...,0,2,--NME--
4,0946-002,West Indian all-rounder Phil Simmons took four...,0,1,--NME--
5,0946-002,West Indian all-rounder Phil Simmons took four...,4,5,--NME--
6,0946-002,West Indian all-rounder Phil Simmons took four...,1,2,--NME--
7,0946-002,West Indian all-rounder Phil Simmons took four...,12,13,--NME--
8,0946-002,West Indian all-rounder Phil Simmons took four...,3,5,--NME--
9,0946-002,West Indian all-rounder Phil Simmons took four...,10,11,--NME--
10,0946-002,West Indian all-rounder Phil Simmons took four...,14,15,--NME--


## Problem 4: Entity linking

Now that we have a method for predicting mention spans, we turn to the task of **entity linking**, which amounts to predicting the knowledge base entity that is referenced by a given mention. In our case, for each span we want to predict the Wikipedia page that this mention references.

Start by extending the generator function that you implemented in Problem&nbsp;2 to labelled spans.

In [168]:
def gold_mentions(df):
    """Yield the gold-standard mentions in a data frame.

    Args:
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        quadruples consisting of the sentence id, start position, end
        position and entity label of each span.
    """
    # TODO: Replace the next line with your own code
    for row in df.itertuples():
        yield (row[1],row[3],row[4],row[5])

In [169]:
list(gold_mentions(df))[:10]

[('0946-000', 0, 1, '--NME--'),
 ('0946-001', 0, 1, '--NME--'),
 ('0946-002', 0, 2, '--NME--'),
 ('0946-002', 0, 1, '--NME--'),
 ('0946-002', 4, 5, '--NME--'),
 ('0946-002', 1, 2, '--NME--'),
 ('0946-002', 12, 13, '--NME--'),
 ('0946-002', 3, 5, '--NME--'),
 ('0946-002', 10, 11, '--NME--'),
 ('0946-002', 14, 15, '--NME--')]

A naive baseline for entity linking on our data set is to link each mention span to the Wikipedia page name that we get when we join the tokens in the span by underscores, as is standard in Wikipedia page names. Suppose, for example, that a span contains the two tokens

    Jimi Hendrix

The baseline Wikipedia page name for this span would be

    Jimi_Hendrix

Implement this naive baseline and evaluate its performance. Print the evaluation measures that you implemented in Problem&nbsp;1.

<div class="alert alert-warning">
    Here and in the remainder of this lab, you should base your entity predictions on the predicted spans that you computed in Problem&nbsp;3.
</div>

In [170]:
# TODO: Write code here to implement the baseline
df4 = df.copy()
for row in df.itertuples():
    s = list(row[2].split())
    label = "_".join(s[row[3]:row[4]])
#     cond =  and df4["end"==row[4]]
    df4.loc[(df4["sentence_id"]==row[1]) & (df4["beg"]==row[3]) , "label"] = label

In [171]:
df4[:10]

,sentence_id,sentence,beg,end,label
1,0946-000,CRICKET - LEICESTERSHIRE TAKE OVER AT TOP AFTE...,0,1,CRICKET
2,0946-001,LONDON 1996-08-30,0,1,LONDON
3,0946-002,West Indian all-rounder Phil Simmons took four...,0,2,West
4,0946-002,West Indian all-rounder Phil Simmons took four...,0,1,West
5,0946-002,West Indian all-rounder Phil Simmons took four...,4,5,Simmons
6,0946-002,West Indian all-rounder Phil Simmons took four...,1,2,Indian
7,0946-002,West Indian all-rounder Phil Simmons took four...,12,13,Leicestershire
8,0946-002,West Indian all-rounder Phil Simmons took four...,3,5,Phil_Simmons
9,0946-002,West Indian all-rounder Phil Simmons took four...,10,11,Friday
10,0946-002,West Indian all-rounder Phil Simmons took four...,14,15,Somerset


## Problem 5: Extending the training data using the knowledge base

State-of-the-art approaches to entity linking exploit information in knowledge bases. In our case, where Wikipedia is the knowledge base, one particularly useful type of information are links to other Wikipedia pages. In particular, we can interpret the anchor texts (the highlighted texts that you click on when you click on a link) as mentions of the entities (pages) that they link to. This allows us to harvest long lists over mention–entity pairings.

The following cell loads a data frame summarizing anchor texts and page references harvested from the first paragraphs of the English Wikipedia. The data frame also contains all entity mentions in the training data (but not the development or the test data).

In [110]:
with bz2.open("kb.tsv.bz2", 'rb') as source:
    df_kb = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

To understand what information is available in this data, the following cell shows the entry for the anchor text `Sweden`.

In [124]:
df_kb.loc[df_kb["mention"] == "Sweden"]

,mention,entity,prob
17436,Sweden,Sweden,0.985768
17437,Sweden,Sweden_national_football_team,0.014173
17438,Sweden,Sweden_men's_national_ice_hockey_team,0.000059


As you can see, each row of the data frame contains a pair $(m, e)$ of a mention $m$ and an entity $e$, as well as the conditional probability $P(e|m)$ for mention $m$ referring to entity $e$. These probabilities were estimated based on the frequencies of mention–entity pairs in the knowledge base. The example shows that the anchor text &lsquo;Sweden&rsquo; is most often used to refer to the entity [Sweden](http://en.wikipedia.org/wiki/Sweden), but in a few cases also to refer to Sweden&rsquo;s national football and ice hockey teams. Note that references are sorted in decreasing order of probability, so that the most probable pairing stands first.

Implement an entity linking method that resolves each mention to the most probable entity in the data frame. If the mention is not included in the data frame, you can predict the generic label `--NME--`. Print the precision, recall, and F1 of your method using the function that you implemented for Problem&nbsp;1.

In [151]:
# TODO: Write code here to implement the "most probable entity" method.
def most_probable_entity(df, ent):
    mentions = df.loc[df["mention"]==ent].sort_values(by=["prob"], ascending=False)
    if mentions.shape[0]>0:
        m = mentions.iloc[0,1]
        return m
    else:
        return "--NME--"

# most_probable_entity(df_kb, "Sweden")
pred5 = [most_probable_entity(df_kb, ent) for ent in df_kb.mention]

In [155]:
evaluation_report(set(pred5), set(df_kb.mention))

{0.03534273906227733, 0.058041112454655375, 0.16223288268643699}

## Problem 6: Context-sensitive disambiguation

Consider the entity mention &lsquo;Lincoln&rsquo;. The most probable entity for this mention turns out to be [Lincoln, Nebraska](http://en.wikipedia.org/Lincoln,_Nebraska); but in pages about American history, we would be better off to predict [Abraham Lincoln](http://en.wikipedia.org/Abraham_Lincoln). This suggests that we should try to disambiguate between different entity references based on the textual context on the page from which the mention was taken. Your task in this last problem is to implement this idea.

Set up a dictionary that contains, for each mention $m$ that can refer to more than one entity $e$, a separate Naive Bayes classifier to predict the correct entity $e$, given the textual context of the mention. As the prior probabilities of the classifier, choose the probabilities $P(e|m)$ that you used in Problem&nbsp;5. To let you estimate the context-specific probabilities, we have compiled a data set with mention contexts:

In [140]:
with bz2.open("contexts.tsv.bz2") as source:
    df_contexts = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

This data frame contains, for each ambiguous mention $m$ and each knowledge base entity $e$ to which this mention can refer, up to 100 randomly selected contexts in which $m$ is used to refer to $e$. For this data, a **context** is defined as a bag of words containing the mention itself, as well as the 5 tokens to the left and the 5 tokens to the right of the mention. Here are a few examples:

In [142]:
df_contexts.head()

,mention,entity,context
0,1970,UEFA_Champions_League,Cup twice the first in 1970 and the second in ...
1,1970,FIFA_World_Cup,America 1975 and during the 1970 and 1978 Worl...
2,1990 World Cup,1990_FIFA_World_Cup,Manolo represented Spain at the 1990 World Cup
3,1990 World Cup,1990_FIFA_World_Cup,Hašek represented Czechoslovakia at the 1990 W...
4,1990 World Cup,1990_FIFA_World_Cup,renovations in 1989 for the 1990 World Cup The...


From this data frame, it is easy to select the data that you need to train the classifiers – the contexts and corresponding entities for all mentions. To illustrate this, the following cell shows how to select all contexts that belong to the mention &lsquo;Lincoln&rsquo;:

In [143]:
df_contexts.context[df_contexts.mention == "Lincoln"]

41465    Nebraska Concealed Handgun Permit In Lincoln m...
41466    Lazlo restaurants are located in Lincoln and O...
41467    California Washington Overland Park Kansas Lin...
41468    City Missouri Omaha Nebraska and Lincoln Nebra...
41469    by Sandhills Publishing Company in Lincoln Neb...
                               ...                        
41609                                Lincoln Leyton Orient
41610               English division three Swansea Lincoln
41611    league membership narrowly edging out Lincoln ...
41612                                    Lincoln Cambridge
41613                                              Lincoln
Name: context, Length: 149, dtype: object

Implement the context-sensitive disambiguation method and evaluate its performance. Here are some more hints that may help you along the way:

**Hint 1:** The prior probabilities for a Naive Bayes classifier can be specified using the `class_prior` option. You will have to provide the probabilities in the same order as the alphabetically sorted class (entity) names.

**Hint 2:** To tune the performance of your method, you can try to tweak the behaviour of the vectorizer (for example, should it apply lowercasing or not?) and the width of the window from which you are extracting context tokens at prediction time.

**Hint 3:** Not all mentions in the knowledge base are ambiguous, and therefore not all mentions have context data. If a mention has only one possible entity, pick that one. If a mention has no entity at all, predict the `--NME--` label.

In [379]:
# TODO: Write code here to implement the context-sensitive disambiguation method
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [380]:
men = df_contexts["mention"].unique()
NBC = {}
for i in men:
    cont = df_contexts.context[df_contexts.mention == i]
    enti = df_contexts.entity[df_contexts.mention == i]
    pri = df_kb.loc[df_kb.mention == i, :].sort_values(by="entity")["prob"].values
    pipeline = Pipeline([
        ('vect', TfidfVectorizer()),
        ('model', MultinomialNB(class_prior = pri)),
    ])
    pred=pipeline.fit(cont,enti)
    NBC[i] = pred

In [381]:
labels = list()
for row in df4.itertuples():
    sent = row[2].split()
    beg = row[3]
    end = row[4]
    men = row[5]
    if men in NBC:
        clf = NBC[men]
        label = clf.predict([row[2]])[0]
    elif men in list(df_kb["mention"]):
        label = df_kb.loc[df_kb.mention==men,"entity"].values[0]
    else:
        label = "--NME--"
    labels.append(label)

In [382]:
df5 = df.copy()
df5["label"]=labels
pred5 = set(row for row in gold_mentions(df5))
gold = set(row for row in gold_mentions(df_dev))
evaluation_report(gold, pred5)


{0.3766686012768427, 0.4053399953157936, 0.4387358458678384}

You should expect to see a small (around 1&nbsp;unit) increase in both precision, recall, and F1. Published systems report a larger impact of context-sensitive disambiguation. Feel free to think about what could explain the relatively minor impact that we see here!

**This was the last lab in the Text Mining course. Well done!**

<div class="alert alert-info">
    Please read the section ‘General information’ on the ‘Labs’ page of the course website before submitting this notebook!
</div>